# S3

In [ ]:
import boto3
s3_client = boto3.client("s3")

In [ ]:
sufix = "rmc-2023"

In [ ]:
bucket_name = f"aws-workshop-dev-{sufix}"
bucket_name

In [ ]:
buckets = s3_client.list_buckets()
buckets

## GET

In [ ]:
buckets.get("Buckets")

In [ ]:
names = [bucket.get("Name") for bucket in buckets.get("Buckets")]
names

In [ ]:
#bucket_name = ""
objects = s3_client.list_objects(Bucket=bucket_name)
objects.get("Contents") or f"Bucket {bucket_name} is empty"

In [ ]:
for file in objects.get("Contents"):
    print(file.get("Key"))

## CREATE

In [ ]:
import io
from reportlab.pdfgen import canvas
from random import randint


params = {"first_name": "Scarlett", "last_name": "Johansson"}


def generate_pdf(params):
    buffer = io.BytesIO()
    p = canvas.Canvas(buffer)
    p.drawString(100, 100, f"Name: {params.get('first_name')}")
    p.drawString(100, 150, f"Last Name: {params.get('last_name')}")
    p.save()
    return buffer


try:
    s3 = boto3.resource("s3")
    object_key = f"pdfs/my_pdf{randint(1,100)}.pdf"
    s3.Object(bucket_name, object_key).put(Body=generate_pdf(params))
    print(f"El archivo {object_key} fue cargado con exito")
except Exception as e:
    print(f"Ocurrió un error al cargar el archivo en S3: {str(e)}")

In [ ]:
from random import randint

try:
    source_path = "dummy/important_file.txt"
    name_given = f"roberto/idk_{randint(10,200)}.txt"
    s3_client.upload_file(source_path, bucket_name, name_given)
    print(
        f"El archivo {name_given} cuyo archivo de origen es {source_path} fue cargado con exito")
except Exception as e:
    print(f"Ocurrió un error al cargar el archivo en S3: {str(e)}")

In [ ]:
s3_client.upload_file?

## DELETE

In [ ]:
folder_name = "pdf"

objects = s3_client.list_objects(Bucket=bucket_name, Prefix=folder_name)

if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        print(f"Eliminando objeto: {obj['Key']}")
else:
    print(f"No se encontraron objetos en la carpeta: {folder_name}")


# SQS

In [ ]:
import boto3
sqs = boto3.client("sqs")

In [ ]:
sqs.list_queues()

In [ ]:
queue_name = f"aws-workshop-dev-{sufix}"
queue_name

In [ ]:
response = sqs.get_queue_url(QueueName=queue_name)
response

In [ ]:
queue_url = response["QueueUrl"]
queue_url

In [ ]:
import json
from random import randint


param = {
    "type": "Welcome",
    "first_name": "Roberto",
    "last_name": "Madinyá",
    "email": "roberto.madinya@ioet.com",
    "hash": randint(5, 100)
}

def save_in_db(user):
    print(f"{user.get('first_name')} Saved into my db")


def notify_new_user():
    try:
        message = json.dumps(param)
        # message = f"type:WELCOME;user:roberto.madinya@ioet.com;hash:{randint(5,100)}"
        sqs.send_message(QueueUrl=queue_url, MessageBody=message)
        print(f"Mensaje {message} enviado correctamente")
    except Exception as e:
        print(f"Error al enviar el mensaje {e}")


def sign_up():
    save_in_db(param)
    notify_new_user()


sign_up()

In [ ]:
def upload_to_s3():
    print("Saving into S3")

def generate_pdf(message):
    print(f"Generating pdf with this content {message}")
    upload_to_s3()

try:
    response = sqs.receive_message(QueueUrl=queue_url, MaxNumberOfMessages=1)
    if "Messages" in response:
        mensaje = response["Messages"][0]
        body = mensaje["Body"]

        generate_pdf(body)

        receipt_handle = mensaje["ReceiptHandle"]
        sqs.delete_message(QueueUrl=queue_url, ReceiptHandle=receipt_handle)
    else:
        print(f"No se encontraron mensajes en la cola '{queue_name}''.")
except Exception as e:
    print(f"Error al enviar el mensaje {e}")

### DynamoDB

In [ ]:
import boto3
dynamodb = boto3.client("dynamodb")

In [ ]:
table_name = f"aws-workshop-dev-{sufix}"

In [ ]:
response = dynamodb.scan(
    TableName=table_name
)

items = response.get("Items", [])
items

In [ ]:
import json
content = {"genre": "F",
           "first_name":  "Graciela",
           "age": 35
           }
item = {
    "id": {"S": f"{randint(1,200)}"},
    "value": {"S": json.dumps(content)}
}

In [ ]:
item = {
    "id": {"S": f"{randint(1,200)}"},
    "genre": {"S": "M"},
    "first_name": {"S": "Roberto"},
    "age": {"N": "33"}
}

In [ ]:
item = {
    "id": {"S": f"{randint(1,200)}"},
    "first_name": {"S": "Daysi"},
    "genre": {"S": "F"},
    "lastName": {"S": "Orellana"},
    "country": {"S": "EC"},
    "civilState": {"S": "Single"},
}

In [ ]:
item = {
    "id": {"S": f"{randint(1,200)}"},
    "first_name": {"S": "Marcela"},
    "genre": {"S": "F"},
    "country": {"S": "USA"},
    "state": {"S": "UT"},
    "age": {"N": "31"}
}

In [ ]:
try:
    response = dynamodb.put_item(
        TableName=table_name,
        Item=item
    )
    print(f"Elemento {item} creado en la tabla '{table_name}'.")
except Exception as e:
    print(e)

In [ ]:
response = dynamodb.scan(
    TableName=table_name,
    FilterExpression="genre = :genre",
    ExpressionAttributeValues={
        ":genre": {"S": "F"}
    }
)

items = response.get("Items", [])
items

In [ ]:
key = {
    "id": {"S": "26"}
}

response = dynamodb.get_item(
    TableName=table_name,
    Key=key
)

item = response.get("Item") or "Not found"
item

In [ ]:
key = {
    "id": {"S": "146"}
}
try:
    response = dynamodb.delete_item(
        TableName=table_name,
        Key=key
    )
except Exception as e:
    print(e)

response

In [ ]:
key = {
    "id": {"S": "777"}
}

update_expression = "SET first_name = :first_name"
expression_attribute_values = {
    ":first_name": {"S": "Name modified"}
}
try:
    response = dynamodb.update_item(
        TableName=table_name,
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeValues=expression_attribute_values,
        ReturnValues="UPDATED_NEW"
    )
    updated_item = response.get("Attributes")
    if updated_item:
        print(f"Elemento actualizado en la tabla '{table_name}': {updated_item}")
    else:
        print(f"No se pudo actualizar el elemento.")
except Exception as e:
    print(e)